In [2]:
import zipfile
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
zf = zipfile.ZipFile('/content/drive/My Drive/GL-AIML/assignment/blog-authorship-corpus.zip') 
blog_df = pd.read_csv(zf.open('blogtext.csv'))

In [8]:
# original data frame
blog_df.shape

(681284, 7)

In [0]:
# crop 30k dataset of this
blog_df=df.iloc[:30000,0:]

In [10]:
blog_df.shape


(30000, 7)

**Preprocess rows of the “text” column**

In [11]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('wordnet')
import re
import unicodedata
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# remove unwanted chars
# convert to lowercase
# remove unwanted spaces
# remove stop words
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 

def preprocess_text(corpus, text_lower_case=True, 
                      special_char_removal=True, stopword_removal=True, remove_digits=True):    
    normalized_text = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits) 

        if stopword_removal:
            doc = remove_stopwords(doc)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)            
        normalized_text.append(doc)
        
    return normalized_text

def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text):  
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]   
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)                 
    return ' '.join(filtered_sentence)

In [13]:
# test 
remove_stopwords("sandipan laptop complex this is a nice day")

'sandipan laptop complex nice day'

In [14]:
# test 
print(preprocess_text(["sandipan UPPER laptop complex this is a nice day","abc def 123 $%% ddd", "   def 123"]))

['sandipan upper laptop complex nice day', 'abc def ddd', 'def']


In [0]:
blog_df['clean_text'] = preprocess_text(blog_df['text'])

In [16]:
blog_df.head()

,id,gender,age,topic,sign,date,text,clean_text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...


**Merge Columns**

In [0]:
blog_df['label']=blog_df['gender'].astype(str)+','+blog_df['age'].astype(str)+','+blog_df['topic'].astype(str)+','+blog_df['sign'].astype(str)

In [0]:
def tostr(label):
  label=str(label)
  return label.split(',')

blog_df['label']= blog_df['label'].map(lambda x: tostr(x))

In [20]:
blog_df.head()

,id,gender,age,topic,sign,date,text,clean_text,label
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing testing,"[male, 15, Student, Leo]"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


In [0]:
blog_df_cleaned = blog_df.filter(['clean_text','label'])

In [22]:
blog_df_cleaned.head()

,clean_text,label
0,info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


In [23]:
blog_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clean_text  30000 non-null  object
 1   label       30000 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB


**Split data into Training and Test**

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(blog_df_cleaned.clean_text, blog_df_cleaned.label,test_size=0.30, random_state=123)

In [25]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(21000,)
(21000,)
(9000,)
(9000,)


**Vectorize the features**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
cvect = CountVectorizer(ngram_range=(1,2))

In [0]:
# create document-term matrices
X_train_ct = cvect.fit_transform(X_train)
X_test_ct = cvect.transform(X_test)


In [29]:
# vocab length 
len(cvect.vocabulary_)

1432467

In [30]:
cvect.vocabulary_

{'urllink': 1320064,
 'dvd': 345081,
 'rewinder': 1034446,
 'found': 452099,
 'pretty': 963253,
 'hilarious': 565224,
 'remember': 1020628,
 'joking': 643019,
 'rewinding': 1034449,
 'dvds': 345322,
 'long': 723691,
 'time': 1262182,
 'ago': 20372,
 'rentals': 1023514,
 'stores': 1182904,
 'first': 434351,
 'started': 1169799,
 'carry': 178864,
 'rented': 1023522,
 'one': 873740,
 'ol': 871206,
 'vhs': 1337500,
 'please': 940031,
 'kind': 658707,
 'rewind': 1034433,
 'sticker': 1177040,
 'knows': 668130,
 'many': 756016,
 'people': 912727,
 'actually': 10056,
 'bitched': 123805,
 'watched': 1358251,
 'used': 1328803,
 'back': 85787,
 'scan': 1067716,
 'button': 162282,
 'pity': 932067,
 'dumbasses': 343466,
 'double': 331405,
 'speed': 1158205,
 'early': 346605,
 'days': 287353,
 'players': 938541,
 'thanks': 1236838,
 'theres': 1241315,
 'longer': 724897,
 'need': 833890,
 'waste': 1356901,
 'player': 938320,
 'thats': 1237812,
 'lost': 730570,
 'could': 254048,
 'watching': 1358884,


In [31]:
print(X_train_ct[600])
print(X_train_ct.shape)

  (0, 1320064)	1
  (0, 873740)	1
  (0, 940031)	1
  (0, 1004372)	1
  (0, 501297)	1
  (0, 128263)	1
  (0, 598549)	1
  (0, 630840)	1
  (0, 417947)	2
  (0, 1252675)	1
  (0, 1143168)	1
  (0, 749882)	1
  (0, 359632)	1
  (0, 1123844)	1
  (0, 455945)	1
  (0, 51120)	1
  (0, 1272256)	1
  (0, 392796)	1
  (0, 970035)	1
  (0, 129900)	1
  (0, 193668)	1
  (0, 986926)	2
  (0, 69544)	1
  (0, 600777)	1
  (0, 750066)	1
  :	:
  (0, 1203400)	1
  (0, 631781)	1
  (0, 1096222)	1
  (0, 986951)	1
  (0, 70220)	1
  (0, 1323697)	1
  (0, 128375)	1
  (0, 129931)	1
  (0, 193689)	1
  (0, 501673)	1
  (0, 221121)	1
  (0, 1272829)	1
  (0, 970066)	1
  (0, 411276)	1
  (0, 1203406)	1
  (0, 972865)	1
  (0, 1123875)	1
  (0, 51543)	1
  (0, 987179)	1
  (0, 456141)	1
  (0, 359853)	1
  (0, 392993)	1
  (0, 418973)	1
  (0, 1253800)	1
  (0, 1143461)	1
(21000, 1432467)


In [32]:
y_train[0]

['male', '15', 'Student', 'Leo']

In [33]:
y_train.head(2)

16395    [male, 27, indUnk, Sagittarius]
21448          [female, 38, Arts, Aries]
Name: label, dtype: object

**Create a dictionary to get the count of every label**

In [42]:
# test

",".join([str(elem).replace("['","").replace("']","").replace(" '","").replace("'","") for elem in y_train])

# ",".join(y_train).split(",")
# ','.join([str(elem) for elem in y_train[0]]) 

'male,27,indUnk,Sagittarius,female,38,Arts,Aries,male,16,Student,Pisces,female,23,Student,Pisces,male,35,Technology,Aries,female,25,indUnk,Aquarius,female,23,indUnk,Cancer,male,16,indUnk,Sagittarius,male,17,indUnk,Gemini,female,15,indUnk,Aries,female,24,Law,Leo,female,27,Marketing,Aquarius,female,17,Student,Capricorn,male,34,Education,Pisces,female,27,Marketing,Aquarius,male,14,indUnk,Scorpio,female,27,indUnk,Taurus,female,26,indUnk,Aquarius,female,23,indUnk,Capricorn,male,35,Technology,Aries,male,27,Technology,Virgo,female,24,indUnk,Aries,male,16,Student,Cancer,male,17,Student,Aquarius,male,23,Construction,Leo,male,16,indUnk,Cancer,female,24,Arts,Taurus,female,15,Student,Aries,male,23,Communications-Media,Virgo,female,27,Government,Leo,male,36,Fashion,Aries,male,35,Technology,Aries,male,33,Marketing,Pisces,male,27,Religion,Aquarius,male,33,Automotive,Gemini,female,13,Student,Sagittarius,male,25,indUnk,Libra,female,15,Student,Pisces,female,23,Student,Pisces,female,17,indUnk,Aquarius,fe

In [43]:
from collections import Counter

countofeverylabel = Counter(",".join([str(elem).replace("['","").replace("']","").replace(" '","").replace("'","") for elem in y_train]).split(",")).items()
countofeverylabel

dict_items([('male', 11220), ('27', 2165), ('indUnk', 8506), ('Sagittarius', 2070), ('female', 9780), ('38', 97), ('Arts', 768), ('Aries', 4843), ('16', 1962), ('Student', 3094), ('Pisces', 1732), ('23', 2521), ('35', 1814), ('Technology', 2279), ('25', 961), ('Aquarius', 2004), ('Cancer', 1660), ('17', 2003), ('Gemini', 1050), ('15', 1085), ('24', 2219), ('Law', 45), ('Leo', 1682), ('Marketing', 238), ('Capricorn', 971), ('34', 651), ('Education', 796), ('14', 817), ('Scorpio', 1580), ('Taurus', 1350), ('26', 1169), ('Virgo', 1031), ('Construction', 18), ('Communications-Media', 596), ('Government', 294), ('36', 1244), ('Fashion', 1145), ('33', 977), ('Religion', 128), ('Automotive', 86), ('13', 378), ('Libra', 1027), ('Manufacturing', 156), ('Engineering', 527), ('45', 54), ('Banking', 68), ('48', 185), ('LawEnforcement-Security', 65), ('BusinessServices', 162), ('Transportation', 31), ('39', 105), ('Accounting', 43), ('Internet', 865), ('37', 134), ('Maritime', 38), ('40', 126), ('N

**Transform the labels**

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabelbin = MultiLabelBinarizer()
y_train_m=multilabelbin.fit_transform(y_train)
y_test_m=multilabelbin.transform(y_test)

In [45]:
y_test

25665                [male, 33, indUnk, Pisces]
16464           [male, 27, indUnk, Sagittarius]
22386         [female, 14, indUnk, Sagittarius]
10149               [male, 17, Student, Cancer]
8729             [male, 27, Technology, Pisces]
                          ...                  
17548              [female, 15, Student, Virgo]
11459    [female, 23, BusinessServices, Cancer]
6608                 [male, 36, Fashion, Aries]
2414              [male, 35, Technology, Aries]
14757                 [male, 23, Internet, Leo]
Name: label, Length: 9000, dtype: object

In [46]:
print(y_train_m[0])
print(y_train_m.shape)

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 1]
(21000, 78)


In [47]:
# total number of classes 
multilabelbin.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', 'Accounting', 'Advertising', 'Agriculture',
       'Aquarius', 'Architecture', 'Aries', 'Arts', 'Automotive',
       'Banking', 'Biotech', 'BusinessServices', 'Cancer', 'Capricorn',
       'Chemicals', 'Communications-Media', 'Construction', 'Consulting',
       'Education', 'Engineering', 'Fashion', 'Gemini', 'Government',
       'HumanResources', 'Internet', 'InvestmentBanking', 'Law',
       'LawEnforcement-Security', 'Leo', 'Libra', 'Manufacturing',
       'Maritime', 'Marketing', 'Military', 'Museums-Libraries',
       'Non-Profit', 'Pisces', 'Publishing', 'RealEstate', 'Religion',
       'Sagittarius', 'Science', 'Scorpio', 'Sports-Recreation',
       'Student', 'Taurus', 'Technology', 'Telecommunications',
       'Transportation', 'Virgo', 'female', 'indUnk', 'male'],
      dtype=object)

**Choose a classifier**

In [48]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
ovrclassifier = OneVsRestClassifier(LogisticRegression(solver='lbfgs',max_iter=500))
ovrclassifier

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=500,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [49]:
ovrclassifier.fit(X_train_ct,y_train_m)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=500,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
y_pred=ovrclassifier.predict(X_test_ct)

In [51]:
print(y_pred.shape)
print(y_test_m.shape)

(9000, 78)
(9000, 78)


**Fit the classifier, make predictions and get the accuracy**

In [0]:
from sklearn.metrics import average_precision_score, recall_score,f1_score,accuracy_score

In [56]:
print("F1 Score ", f1_score(y_test_m,y_pred,average='weighted'))
print("Accuracy", accuracy_score(y_test_m,y_pred))
print("Precision: ", average_precision_score(y_test_m, y_pred,average='micro'))
print("Recall: ",recall_score(y_test_m,y_pred,average='micro'))

F1 Score  0.4596031183828673
Accuracy 0.12577777777777777
Precision:  0.3136222587315989
Recall:  0.384


**Print true label and predicted label for any five examples**

In [0]:
y_pred_m =multilabelbin.inverse_transform(y_pred)
y_test_inv=multilabelbin.inverse_transform(y_test_m)

In [64]:
print(y_pred_m[87])
print(y_test_inv[87])

('female',)
('26', 'Scorpio', 'female', 'indUnk')


In [69]:
print(y_pred_m[270])
print(y_test_inv[270])

('female',)
('15', 'Taurus', 'female', 'indUnk')


In [71]:
print(y_pred_m[570])
print(y_test_inv[570])

('17', 'Aquarius', 'male')
('17', 'Aquarius', 'Internet', 'male')


In [74]:
print(y_pred_m[8999])
print(y_test_inv[8999])

('male',)
('23', 'Internet', 'Leo', 'male')
